In [3]:
from google.colab import drive
drive.mount('/content/drive/')
2+2

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


4

## Import 

In [0]:
import numpy as np
import pandas as pd
import warnings
from lightgbm import LGBMClassifier
from sklearn.model_selection import cross_val_score
import requests
import json
from tqdm import tqdm_notebook as tqdm
warnings.filterwarnings('ignore')

In [6]:
!ls

drive  sample_data


## Read extracted data 

In [0]:
train_home = pd.read_csv("/content/drive/My Drive/data_sber/train_extracted.csv")
train_final = pd.read_csv("/content/drive/My Drive/data_sber/train_extracted_final.csv")
test_final = pd.read_csv('/content/drive/My Drive/data_sber/test_extracted_final.csv')

In [19]:
train_home.shape, train_final.shape

((99869, 809), (40395, 809))

## Def models 

In [0]:
lgb_top = LGBMClassifier(learning_rate=0.025, n_estimators=2500,num_laeves=69,reg_lambda=11, random_seed=1007)

In [0]:
lgb_test = LGBMClassifier(learning_rate=0.015, n_estimators=2700,num_laeves=74,reg_lambda=11, random_seed=1007)

## Prepare data 

In [0]:
train = pd.concat([train_home, train_final])
train = train.reset_index(drop=True)
train = train.drop('id',axis=1)

In [0]:
y = train['skilled']
X = train.drop('skilled', axis=1)

## Fit models 

In [30]:
%time
lgb_top.fit(X, y)

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 8.11 µs


LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.025, max_depth=-1,
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=2500, n_jobs=-1, num_laeves=69, num_leaves=31,
        objective=None, random_seed=1007, random_state=None, reg_alpha=0.0,
        reg_lambda=11, silent=True, subsample=1.0,
        subsample_for_bin=200000, subsample_freq=0)

## Predict 

In [0]:
test_index = test_final['id']
test_data = test_final.drop('id',axis=1)

In [0]:
pred = lgb_top.predict_proba(test_data)

In [0]:
proba = pred[:,1]

In [35]:
proba

array([0.84728764, 0.00801701, 0.90702956, ..., 0.38974594, 0.31637885,
       0.07052642])

In [37]:
predict.shape, test_final.shape

((15835, 1), (15835, 808))

In [0]:
predict = pd.DataFrame({'skilled_prob': proba}, index=test_index)
predict.to_csv('lgb_pred.csv')